# Module 3 : Wrap up Quizzzzzzz

Import des données :

In [1]:
import pandas as pd

penguins = pd.read_csv("../datasets/penguins.csv")

columns = ["Body Mass (g)", "Flipper Length (mm)", "Culmen Length (mm)"]
target_name = "Species"

# Remove lines with missing values for the columns of interestes
penguins_non_missing = penguins[columns + [target_name]].dropna()

data = penguins_non_missing[columns]
target = penguins_non_missing[target_name]

In [4]:
data

,Body Mass (g),Flipper Length (mm),Culmen Length (mm)
0,3750.0,181.0,39.1
1,3800.0,186.0,39.5
2,3250.0,195.0,40.3
4,3450.0,193.0,36.7
5,3650.0,190.0,39.3
...,...,...,...
339,4000.0,207.0,55.8
340,3400.0,202.0,43.5
341,3775.0,193.0,49.6
342,4100.0,210.0,50.8


In [5]:
target

0            Adelie Penguin (Pygoscelis adeliae)
1            Adelie Penguin (Pygoscelis adeliae)
2            Adelie Penguin (Pygoscelis adeliae)
4            Adelie Penguin (Pygoscelis adeliae)
5            Adelie Penguin (Pygoscelis adeliae)
                         ...                    
339    Chinstrap penguin (Pygoscelis antarctica)
340    Chinstrap penguin (Pygoscelis antarctica)
341    Chinstrap penguin (Pygoscelis antarctica)
342    Chinstrap penguin (Pygoscelis antarctica)
343    Chinstrap penguin (Pygoscelis antarctica)
Name: Species, Length: 342, dtype: object

In [8]:
target.value_counts()

Adelie Penguin (Pygoscelis adeliae)          151
Gentoo penguin (Pygoscelis papua)            123
Chinstrap penguin (Pygoscelis antarctica)     68
Name: Species, dtype: int64

#### Question 1 : Select the correct assertions from the following proposals.

 a) The problem to be solved is a regression problem  
 b) The problem to be solved is a binary classification problem (exactly 2 possible classes)  
 c) The problem to be solved is a multiclass classification problem (more than 2 possible classes)  
 d) The proportions of the class counts are balanced: there are approximately the same number of rows for each class  
 e) The proportions of the class counts are imbalanced: some classes have more than twice as many rows than others)  
 f) The input features have similar dynamic ranges (or scales)

Je dirais : c) et e)

#### Question 2 : Let's now consider the following pipeline:

```
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("classifier", KNeighborsClassifier(n_neighbors=5)),
])
```

Evaluate the pipeline using 10-fold cross-validation using the balanced-accuracy scoring metric to choose the correct statements.

Use sklearn.model_selection.cross_validate with scoring="balanced_accuracy". Use model.get_params() to list the parameters of the pipeline and use model.set_params(param_name=param_value) to update them.

For this question, we consider a model is better if its mean cross-validation score is larger than the mean plus the standard deviation of the cross-validation score of the model that we compare to.

 a) The average cross-validated test balanced_accuracy of the above pipeline is between 0.9 and 1.0  
 b) The average cross-validated test balanced_accuracy of the above pipeline is between 0.8 and 0.9  
 c) The average cross-validated test balanced_accuracy of the above pipeline is between 0.5 and 0.8  
 d) Using n_neighbors=5 is much better than n_neighbors=51  
 e) Preprocessing with StandardScaler is much better than using the raw features (with n_neighbors=5)

 Du coup a) et e)

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("classifier", KNeighborsClassifier(n_neighbors=5)),
])

## Calcul du modèle initial
cv_results = cross_validate(model, data, target, scoring="balanced_accuracy", cv=10)
print(f"Score du modèle : {cv_results['test_score'].mean():.3f} +/- {cv_results['test_score'].std():.3f}")

## Calcul du modèle avec changement du param n_neighbors
nb_neighbors = 51
model.set_params(classifier__n_neighbors=nb_neighbors)
cv_results = cross_validate(model, data, target, scoring="balanced_accuracy", cv=10)
print(f"Score du modèle avec {nb_neighbors} voisins : {cv_results['test_score'].mean():.3f} +/- {cv_results['test_score'].std():.3f}")

## Calcul du modèle sans StandardScaler
model.set_params(preprocessor=None, classifier__n_neighbors=5)
cv_results = cross_validate(model, data, target, scoring="balanced_accuracy", cv=10)
print(f"Score du modèle sans normalisation des données : {cv_results['test_score'].mean():.3f} +/- {cv_results['test_score'].std():.3f}")



Score du modèle : 0.952 +/- 0.040
Score du modèle avec 51 voisins : 0.942 +/- 0.039
Score du modèle sans normalisation des données : 0.740 +/- 0.087


#### Question 3 : We will now study the impact of different preprocessors defined in the list below:

```
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer


all_preprocessors = [
    None,
    StandardScaler(),
    MinMaxScaler(),
    QuantileTransformer(n_quantiles=100),
    PowerTransformer(method="box-cox"),
]
```

The Box-Cox method is common preprocessing strategy for positive values. The other preprocessors work both for any kind of numerical features. If you are curious to read the details about those method, please feel free to read them up in the preprocessing chapter of the scikit-learn user guide but this is not required to answer the quiz questions.

Use sklearn.model_selection.GridSearchCV to study the impact of the choice of the preprocessor and the number of neighbors on the 10-fold cross-validated balanced_accuracy metric. We want to study the n_neighbors in the range [5, 51, 101] and preprocessor in the range all_preprocessors.

Let us consider that a model is significantly better than another if the its mean test score is better than the mean test score of the alternative by more than the standard deviation of its test score.

Which of the following statements hold:

 a) The best model with StandardScaler is significantly better than using any other processor  
 b) Using any of the preprocessors has always a better ranking than using no processor, irrespective of the value of n_neighbors  
 c) The model with n_neighbors=5 and StandardScaler is significantly better than the model with n_neighbors=51 and StandardScaler.  
 d) The model with n_neighbors=51 and StandardScaler is significantly better than the model with n_neighbors=101 and StandardScaler.




In [55]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV

all_preprocessors = [
    None,
    StandardScaler(),
    MinMaxScaler(),
    QuantileTransformer(n_quantiles=100),
    PowerTransformer(method="box-cox"),
]
neighbors = [5, 51, 101]

param_grid = {
    'classifier__n_neighbors': neighbors,
    'preprocessor': all_preprocessors
}

model_grid_search = GridSearchCV(model, param_grid=param_grid, scoring="balanced_accuracy", n_jobs=2, cv=10)
cv_results = model_grid_search.fit(data, target).cv_results_

results = (pd.DataFrame(cv_results).sort_values(by="mean_test_score", ascending=False))
results = results[
    [c for c in results.columns if c.startswith("param_")]
    + ["mean_test_score", "std_test_score"]
]

results


,param_classifier__n_neighbors,param_preprocessor,mean_test_score,std_test_score
1,5,StandardScaler(),0.952198,0.039902
2,5,MinMaxScaler(),0.947778,0.034268
3,5,QuantileTransformer(n_quantiles=100),0.947094,0.033797
4,5,PowerTransformer(method='box-cox'),0.946960,0.047387
6,51,StandardScaler(),0.941880,0.038905
8,51,QuantileTransformer(n_quantiles=100),0.927277,0.043759
9,51,PowerTransformer(method='box-cox'),0.922833,0.047883
7,51,MinMaxScaler(),0.920293,0.045516
11,101,StandardScaler(),0.876642,0.041618
12,101,MinMaxScaler(),0.862357,0.046244
